In [1]:
from common import *
# Doesn't really work as intended but might be useful for future exploration
class lua_tools:
    _arg = r'(L\d_\d)'
    _n = r'\n\s*'
    _no_scope = r'(?:[\W\w](?!\bif )(?!\belse )(?!\bfunction )(?!\bend\b))'
    _no_vars = r'(?:(?!L\d_\d)[\W\w])'
    move_init_to_usage = re.compile(_arg + r' = {}\n(?!.*\1)(' + _no_scope + r'+?)(?=\n.*\1)')  # L2_1 = {}; ...; thing = L2_1  -> ...; L2_1 = {}; thing = L2_1;
    table_init = re.compile(_arg + r' = \{\}' + _n + r'\1.(\w+) = (.+)')                        # L4_1 = {}; L4_1.foo = bar  ->  L4_1 = {"foo"=bar}
    table_init_2 = re.compile(_arg + r' = \{(.+)\}' + _n + r'\1.(\w+) = (.+)')                  # L4_1 = {"foo"=bar}; L4_1.bar = baz  -> L4_1 =  {"foo"=bar, "bar"=baz}
    temp_register = re.compile(_arg + r' = (\w+)' + _n + r'\1 = \1.(\w+)')                      # L4_2 = GadgetState; L4_2 = L4_2.Action01  ->  L4_2 = GadgetState.Action01
    temp_register_2 = re.compile(_arg + r' = (.*)' + _n + r'(\w.*) = \1(?![\n.]*=.*\1)')
    find_usage = re.compile(_arg + r' = (' + _no_vars + r'+)\n(?!.*\1)(' + _no_scope + r'+?)\n(.*) = \1')  # L2_1 = {foo}; ...; thing = L2_1  -> ...; L2_1 = {foo}; thing = L2_1;
    # array_init_1 = re.compile(_arg + r' = {}' + _n + r'\1\[1\] = (.*)')  # L2_1 = {}; L2_1[1] = foo  ->  L2_1 = {foo}
    array_init = re.compile(_arg + r' = \{\}' + _n + r'\1(\[\d+\]) = (.+)')                        # L4_1 = {}; L4_1[foo] = bar  ->  L4_1 = {[foo]=bar}
    array_init_2 = re.compile(_arg + r' = \{(.+)\}' + _n + r'\1(\[\d+\]) = (.+)')                  # L4_1 = {"foo"=bar}; L4_1[bar] = baz  -> L4_1 =  {"foo"=bar, [bar]=baz}
    func_names = re.compile(r'function (L\d_\d)([\W\w]+?)\n(\w+) = \1')  # function L2_1(...; global_thing = L2_1  -> func global_thing(...
    @staticmethod
    def match_x_before_y(x: re.match, y: re.match) -> bool:
        if y is None:
            return True
        if x is None:
            return False
        return x.start() < y.start()

    @classmethod
    def _remove_temp_register_2(cls, lua: str):
        # L4_1 = foo; L5_1.bar = L4_1  ->  L5_1.bar = foo
        # We need to ensure that L4_1's foo value isn't used again
        hits = 0
        output = lua
        snipped_chars = 0
        for match in cls.temp_register_2.finditer(lua):
            start, end = match.span()
            register_name, value, dest = match.group(1, 2, 3)
            next_reassign = re.search(f'{register_name}.*=', lua[end:])
            next_use = re.search(f'=.*{register_name}', lua[end:])
            if lua_tools.match_x_before_y(next_reassign, next_use):
                newstr = f'{dest} = {value}'
                output = output[:start-snipped_chars] + newstr + output[end-snipped_chars:]  # output shrinks relative to lua with each replacement
                snipped_chars += (end - start) - len(newstr)
                hits += 1
        return output, hits

    @classmethod
    def _remove_temp_register_3(cls, lua: str):
        # L4_1 = foo; ...; L5_1 = L4_1  ->  ...; L5_1 = foo
        # We need to ensure that L4_1's foo value isn't used again
        hits = 0
        output = lua
        snipped_chars = 0
        progress = 0
        while match := cls.find_usage.search(lua, progress):
            start, end = match.span()
            register_name, value, contained_lines, dest = match.group(1, 2, 3, 4)
            next_reassign = re.search(f'{register_name}.*=', lua[end:])
            next_use = re.search(f'=.*{register_name}', lua[end:])
            if lua_tools.match_x_before_y(next_reassign, next_use):
                newstr = f'{contained_lines}\n{dest} = {value}'
                output = output[:start-snipped_chars] + newstr + output[end-snipped_chars:]  # output shrinks relative to lua with each replacement
                snipped_chars += (end - start) - len(newstr)
                hits += 1
                break  # Making this work multiple times would be very hacky
            progress = match.end(2)  # Move start of next search to end of the first line of this match
        return output, hits

    subs = {
        'move_init_to_usage': lambda x: lua_tools.move_init_to_usage.subn(r'\2\n\1 = {}', x),
        'table_init': lambda x: lua_tools.table_init.subn(r'\1 = {"\2"=\3}', x),
        'table_init_2': lambda x: lua_tools.table_init_2.subn(r'\1 = {\2, "\3"=\4}', x),
        'array_init': lambda x: lua_tools.array_init.subn(r'\1 = {\2=\3}', x),
        'array_init_2': lambda x: lua_tools.array_init_2.subn(r'\1 = {\2, \3=\4}', x),
        'temp_register': lambda x: lua_tools.temp_register.subn(r'\1 = \2.\3', x),
        'temp_register_2': lambda x: lua_tools._remove_temp_register_2(x),
        'temp_register_3': lambda x: lua_tools._remove_temp_register_3(x),
        'func_names': lambda x: lua_tools.func_names.subn(r'function \3\2', x),
        #'array_init_1': lambda x: lua_tools.array_init_1.subn(r'\1 = {\2}', x),
    }
    @classmethod
    def refactor(cls, lua: str):
        # Perform every substitution in order until a full round achieves no substitutions
        round_hits = -1
        while round_hits != 0:
            round_hits = 0
            for stage in cls.subs.values():
                lua, hits = stage(lua)
                round_hits += hits
        return lua


base_path = 'resources/Scripts/Scene/'
cleanup_files = ['3/scene3_group133007228.lua', '3/scene3_group133007229.lua', '3/scene3_group133007230.lua', '3/scene3_group133103540.lua', '3/scene3_group133104488.lua', '3/scene3_group133104585.lua',]
for cleanup_file in cleanup_files:
    with open_read(base_path + cleanup_file) as f:
        lua = f.read()
    lua = lua_tools.refactor(lua)
    with open_write(cleanup_file.replace('/', '__'), True) as f:
        f.write(lua)
